# Project: Getting Started with Iterative Product Management

The boutique ice cream sandwich startup you've joined, IceCreamSandwi.ch, has gotten by on the personality of it's founder, I. Sandwich (real name?), for too long.  The last 2 quarters of features the team has launched to the website have not landed with customers.

You've decided it's time to step up, lay out a plan for how you're going to identify paths forward which are more likely to succeed than "whatever I. Sandich happened to promise a random person they met at a bar last night".  You pitch the team on an iterative design process which brings qualitative and quantitative techniques together in order to minimize industry standard risk types so that you can move forward confidently.

Someone in your company recently learned about a/b experiments - so they instrumented tracking and ran an experiment based on your founder's unfounded random idea.

Unsurprisingly - the results of the experiment seem to have flopped.  While the total effect isn't known it definitely wasn't a 10x experiment. 

But there's a bright side! 

The data scientist on your team now has some well structured event logs they can dig through in order to surface user personas and a journey.  You tell the team you'd like to collect all of the known assumptions before you launch the next experiment to the site so you have the best chance of success.  You tell the CEO and Board (wait - is it really just their friends from college and mom?) that at the end of your week long design sprint you're going to summarize what's known and propose a path forward which is iterative and gives you the best chance of moving the needle.  


Your CEO gives you the ok if you can help them show the largest possible impact in the shortest time possible.




<!--
%%ulab_page_divider
--><hr/>

## Submission Requirements

Section 1:   
1: Ensure you have well formed event data  
2: Check to see if the previous a/b experiment was valid (95% confidence)  
3: Check if the experiment or control won by performing a chi-square test on outcomes and reporting on if the difference is signifcant.  

Section 2:   
1: Find the actionable attributes of your users upon which to build your personas  
2: Find the different stages of the user's journey  
3: Identify risks or opportunities for your team to address with hypotheses    

Section 3:   
1: Generate well formed hypotheses with clear next steps

<!--
%%ulab_page_divider
--><hr/>

## Offline Instructions

This notebook contains everything you need to complete your analysis and submit your final project.

In case you need to work offline here's what you need to do

1) Take all files starting with `event_logs_part*`  
2) Load them into your analysis tool of choice (R, SAS, Excel, etc)  
3) Complete your analysis  
4) Upload it or your results back with each of the sections and responses clearly labeled  

<!--
%%ulab_page_divider
--><hr/>

## Section 1 - Evaluate previous a/b experiment [STUDENT]

One of your engineers helpfully exported all of the events from the previous a/b experiment for you.  

She sent them over to you broken down by country and explained how to put them all into one dataframe.

1: Ensure you got data  
2: Check to see if the previous a/b experiment was valid (95% confidence)  
3: Check if the experiment or control won by performing a chi-square test on outcomes and reporting on if the difference is signifcant.


In [8]:
import numpy as np
from scipy import stats
import pandas as pd
import glob
import datetime
import itertools

log_files = glob.glob(r'event_logs_part_*.csv')
df = pd.concat((pd.read_csv(f) for f in log_files)).drop(['Unnamed: 0'],axis=1)

## Your Work Goes Here

#1: 

#2: 

#3: 

## Section 1 - Evaluate previous a/b experiment [SOLUTION]

One of your engineers helpfully exported all of the events from the previous a/b experiment for you.  

She sent them over to you broken down by country and explained how to put them all into one dataframe.

1: Ensure you got data  
2: Check to see if the previous a/b experiment was valid (95% confidence)  
3: Check if the experiment or control won by performing a chi-square test on outcomes and reporting on if the difference is signifcant.

In [9]:
import numpy as np
from scipy import stats
import pandas as pd
import glob
import datetime
import itertools

log_files = glob.glob(r'event_logs_part_*.csv')
df = pd.concat((pd.read_csv(f) for f in log_files)).drop(['Unnamed: 0'],axis=1)

In [10]:
#S1_1
df.describe()

,event_uuid,user_uuid,event_time,device_type,session_uuid,experiment_group,user_country,event_page,event_type
count,1252068,1252068,1252068,1252068,1252068,1252068,1252068,1252068,1252068
unique,1252068,81708,526433,4,227564,2,5,4,5
top,62941cb3-b561-4889-9080-2dfc035ffcb9,80450b40-d083-4a2a-aecc-a7d670764729,2019-10-01 03:29:24,mobile_web,27df8dfd-05e1-4be5-b64d-3207563daeab,control,US,item_page,view
freq,1,49,12,627052,10,627723,877653,500191,821287


In [11]:
#S1_2
## Check for duplicate events
df[df.event_uuid.duplicated()]
#df = df[~df.event_uuid.duplicated()]

,event_uuid,user_uuid,event_time,device_type,session_uuid,experiment_group,user_country,event_page,event_type


In [12]:
## Get the number of sessions by experiment or control group
df.groupby('experiment_group').apply(lambda x: x['session_uuid'].nunique()).reset_index(name='count')

,experiment_group,count
0,control,113911
1,experiment,113653


In [13]:
## Ensure the experiment was split evenly between sessions

control_sessions = df.groupby('experiment_group').apply(lambda x: x['session_uuid'].nunique())['control']
experiment_sessions = df.groupby('experiment_group').apply(lambda x: x['session_uuid'].nunique())['experiment']


desired_p = 0.5  
z_score = 1.960  #TODO Student to get the z-score for 95% confidence range?

total_control =  22725 #TODO Student to get the total number of sessions
total_experiment = 22879 #TODO Student to get the total number of sessions


standard_deviation = np.sqrt((0.5*0.5)/(total_control + total_experiment))
margin_error = standard_deviation * z_score

confidence_interval = (desired_p-margin_error, desired_p+margin_error)

p_hat = total_control / (total_control + total_experiment)

print("p_hat:\t\t", p_hat)
print("confidence int:\t", confidence_interval)
print("Continue analysis?:", p_hat > confidence_interval[0] and p_hat < confidence_interval[1])

p_hat:		 0.4983115516182791
confidence int:	 (0.49541093079380116, 0.50458906920619884)
Continue analysis?: True


In [14]:
## Get the number of purchases by experiment or control group
df.groupby('experiment_group')['event_type'].apply(lambda x: (x == 'purchase').sum()).reset_index(name='count')


,experiment_group,count
0,control,1957
1,experiment,1953


In [15]:
control_purchase = df.groupby('experiment_group')['event_type'].apply(lambda x: (x == 'purchase').sum())['control']
experiment_purchase = df.groupby('experiment_group')['event_type'].apply(lambda x: (x == 'purchase').sum())['experiment']


In [18]:
#S1_3
#chisuared test

#P-value here greater than .05 (our test for significance).  This result could be due to randomness

control_stats = [control_purchase, control_sessions - control_purchase]
experiment_stats = [experiment_purchase, experiment_sessions - experiment_purchase]
outcomes =  np.array([control_stats, experiment_stats])
stats.chi2_contingency(outcomes)

(8.3674337178373762e-05,
 0.99270155826395723,
 1,
 array([[   1957.21647537,  111953.78352463],
        [   1952.78352463,  111700.21647537]]))

### Seciton 1 - Solution summary

1: The experiment ran and there is data.  
2: Event data came in and it was bucketed correctly between sessions  
3: But there was no significant change in conversion between the treatment and control.

## Section 2 - Develop Actionable Personas and Journey Steps [STUDENT]

Now you're going to devolp actionable user personas & stages of a user journey.

1: Find the actionable attributes of your users upon which to build your personas  
2: Find the different stages of the user's journey  
3: Identify a risks or opportunities for your team to address with hypotheses  



In [19]:
## Your work goes here

## Section 2 - Develop Actionable Personas and Journey Steps [SOLUTION]

In [21]:
#S2_1
# Going back to the data - the user should be able to identify the ways the user's differ.
# In this case the only two ways are device type and user country
df.describe()

,event_uuid,user_uuid,event_time,device_type,session_uuid,experiment_group,user_country,event_page,event_type
count,1252068,1252068,1252068,1252068,1252068,1252068,1252068,1252068,1252068
unique,1252068,81708,526433,4,227564,2,5,4,5
top,62941cb3-b561-4889-9080-2dfc035ffcb9,80450b40-d083-4a2a-aecc-a7d670764729,2019-10-01 03:29:24,mobile_web,27df8dfd-05e1-4be5-b64d-3207563daeab,control,US,item_page,view
freq,1,49,12,627052,10,627723,877653,500191,821287


In [22]:
#S2_1
# One possible way students might get the values
# This is the minimal effort. This student will have the types,
# but not have the quantities of users in each group.
devices = df.device_type.unique()
countries = df.user_country.unique()

In [23]:
#S2_1
# This and the the following code samples are examples which are 
# slightly better than the first.
# Now the student might see which country or which device type is most imporant, 
# but not have a quantitative reason for the combination of the two.

# Said another way - they may jump to the conclusion that "US" and "Mobile Web" is the right answer.
# Simpsons paradox could trick them into thinking the independent sums are large when combined


df.groupby('user_country')['user_uuid'].nunique().reset_index(name='count')

,user_country,count
0,CN,1591
1,FR,4124
2,JP,2394
3,UK,16316
4,US,57283


In [24]:
#S2_1
# Continuation of last example
df.groupby('device_type')['user_uuid'].nunique().reset_index(name='count')

,device_type,count
0,android,28606
1,desktop_web,20050
2,ios,42289
3,mobile_web,64359


In [25]:
# S2_1 
# In this example the student has moved on to looking for the most impactful area
# They've managed to count the number of sessions at the device / country level.

# If they use this method and then manually get the numbers that's a hard way to do it,
# but the correct answer
#list(itertools.product(devices,countries))

#df['Rank'] = (df['device_type'] + df['user_country']).astype(int).rank(method)
df.groupby(['device_type', 'user_country'])['user_uuid'].nunique()

device_type  user_country
android      CN                558
             FR               1425
             JP                846
             UK               5764
             US              20013
desktop_web  CN                361
             FR               1016
             JP                579
             UK               4030
             US              14064
ios          CN                801
             FR               2121
             JP               1280
             UK               8415
             US              29672
mobile_web   CN               1245
             FR               3285
             JP               1921
             UK              12850
             US              45058
Name: user_uuid, dtype: int64

In [26]:
#S2_2

# This student has made their life really easy.
# They've not only found the number of sessions per device/region combination, 
# but they've also just rank ordered them - so they know where to start
df.groupby(['device_type', 'user_country'])['user_uuid'].nunique().rank(method='dense')

device_type  user_country
android      CN               2.0
             FR               9.0
             JP               5.0
             UK              14.0
             US              18.0
desktop_web  CN               1.0
             FR               6.0
             JP               3.0
             UK              13.0
             US              17.0
ios          CN               4.0
             FR              11.0
             JP               8.0
             UK              15.0
             US              19.0
mobile_web   CN               7.0
             FR              12.0
             JP              10.0
             UK              16.0
             US              20.0
Name: user_uuid, dtype: float64

In [27]:
# S2_2
# One possible way students might get the values
# This is the minimal effort. This student will have the types of pages or events,
# but not have the quantities of users in each group.

pages = df.event_page.unique()

In [28]:
events = df.event_type.unique()

In [29]:
# This and the the following code samples are examples which are 
# slightly better than the first.

# Now the student might see which page or which event type is most imporant, 
# but not have a quantitative reason for the combination of the two.

# Said another way - they may jump to the conclusion that "Item Page" and "View" is the right answer.

# Simpsons paradox could trick them into thinking the independent sums are large when combined
df.groupby('event_page')['event_uuid'].nunique().reset_index(name='count')


,event_page,count
0,home_page,125385
1,item_page,500191
2,marketing_page,375753
3,search_page,250739


In [30]:


df.groupby('event_type')['event_uuid'].nunique().reset_index(name='count')




,event_type,count
0,cart,36709
1,favorite,194856
2,purchase,3910
3,share,195306
4,view,821287


In [31]:
df.groupby(['event_page', 'event_type'])['session_uuid'].nunique()
# In this example the student has moved on to looking for the most impactful area
# They've managed to count the number of sessions at the event page / event type level.

# If they use this method and then manually get the numbers that's a hard way to do it,
# but the correct answer
#list(itertools.product(pages,events))


event_page      event_type
home_page       cart            3364
                favorite       17799
                purchase         328
                share          17772
                view           64938
item_page       cart           11140
                favorite       54707
                purchase        1275
                share          54942
                view          151295
marketing_page  cart            8940
                favorite       44487
                purchase        1046
                share          44807
                view          132963
search_page     cart            6314
                favorite       32559
                purchase         715
                share          32412
                view          106021
Name: session_uuid, dtype: int64

In [32]:
# This student has made their life really easy.
# They've not only found the number of sessions per device/region combination, 
# but they've also just rank ordered them - so they know where to start
df.groupby(['event_page', 'event_type'])['session_uuid'].nunique().rank(method='dense')

event_page      event_type
home_page       cart           5.0
                favorite      10.0
                purchase       1.0
                share          9.0
                view          17.0
item_page       cart           8.0
                favorite      15.0
                purchase       4.0
                share         16.0
                view          20.0
marketing_page  cart           7.0
                favorite      13.0
                purchase       3.0
                share         14.0
                view          19.0
search_page     cart           6.0
                favorite      12.0
                purchase       2.0
                share         11.0
                view          18.0
Name: session_uuid, dtype: float64

In [33]:
# If they use this method and then manually get the numbers that's a hard way to do it,
# but it's still the correct answer

list(itertools.product(pages, events))

[('marketing_page', 'share'),
 ('marketing_page', 'view'),
 ('marketing_page', 'favorite'),
 ('marketing_page', 'cart'),
 ('marketing_page', 'purchase'),
 ('home_page', 'share'),
 ('home_page', 'view'),
 ('home_page', 'favorite'),
 ('home_page', 'cart'),
 ('home_page', 'purchase'),
 ('search_page', 'share'),
 ('search_page', 'view'),
 ('search_page', 'favorite'),
 ('search_page', 'cart'),
 ('search_page', 'purchase'),
 ('item_page', 'share'),
 ('item_page', 'view'),
 ('item_page', 'favorite'),
 ('item_page', 'cart'),
 ('item_page', 'purchase')]

In [38]:
#S2_3 Looking for Opporutnities
# Conversion rate by Region

## This example will return that users in the US don't act any different from users in any other region - on average.
## The student can tell this because the p-value is so large

purchase_by_region =  df.groupby('user_country')['event_type'].apply(lambda x: (x == 'purchase').sum()).reset_index(name='count')
session_by_region =   df.groupby('user_country')['session_uuid'].nunique().reset_index(name='count')


by_region_conversion = purchase_by_region.merge(session_by_region, on="user_country", suffixes=("_purchase", "_sessions") )

purch_CN = by_region_conversion.loc[ by_region_conversion['user_country'] == 'CN', 'count_purchase'].iloc[0]
purch_FR =  by_region_conversion.loc[ by_region_conversion['user_country'] == 'FR', 'count_purchase'].iloc[0] 
purch_JP =  by_region_conversion.loc[ by_region_conversion['user_country'] == 'JP', 'count_purchase'].iloc[0] 
purch_UK =  by_region_conversion.loc[ by_region_conversion['user_country'] == 'UK', 'count_purchase'].iloc[0] 
purch_US =  by_region_conversion.loc[ by_region_conversion['user_country'] == 'US', 'count_purchase'].iloc[0] 

sess_CN = by_region_conversion.loc[ by_region_conversion['user_country'] == 'CN', 'count_sessions'].iloc[0]
sess_FR =  by_region_conversion.loc[ by_region_conversion['user_country'] == 'FR', 'count_sessions'].iloc[0] 
sess_JP =  by_region_conversion.loc[ by_region_conversion['user_country'] == 'JP', 'count_sessions'].iloc[0] 
sess_UK =  by_region_conversion.loc[ by_region_conversion['user_country'] == 'UK', 'count_sessions'].iloc[0] 
sess_US =  by_region_conversion.loc[ by_region_conversion['user_country'] == 'US', 'count_sessions'].iloc[0]

purch_all_region = by_region_conversion['count_purchase'].sum()
sess_all_region  = by_region_conversion['count_sessions'].sum()

us_stats = [purch_US, sess_US - purch_US]
all_region_stats = [purch_all_region, sess_all_region - purch_all_region]
region_outcomes =  np.array([ all_region_stats, us_stats])
stats.chi2_contingency(region_outcomes)



(0.78973631940260369,
 0.37417986862285735,
 1,
 array([[   3946.01636857,  223617.98363143],
        [   2763.98363143,  156633.01636857]]))

In [37]:
# Conversion rate by Device

## This example will return that users on mobile web have a much worse conversion rate than users on any other platform.
## This is an area rife for hypothesis generation.


purchase_by_device =  df.groupby('device_type')['event_type'].apply(lambda x: (x == 'purchase').sum()).reset_index(name='count')
session_by_device =   df.groupby('device_type')['session_uuid'].nunique().reset_index(name='count')
by_device_conversion = purchase_by_device.merge(session_by_device, on="device_type", suffixes=("_purchase", "_sessions") )


purch_android = by_device_conversion.loc[ by_device_conversion['device_type'] == 'android', 'count_purchase'].iloc[0]
purch_web =  by_device_conversion.loc[ by_device_conversion['device_type'] == 'desktop_web', 'count_purchase'].iloc[0] 
purch_ios =  by_device_conversion.loc[ by_device_conversion['device_type'] == 'ios', 'count_purchase'].iloc[0] 
purch_mweb =  by_device_conversion.loc[ by_device_conversion['device_type'] == 'mobile_web', 'count_purchase'].iloc[0] 

sess_android = by_device_conversion.loc[ by_device_conversion['device_type'] == 'android', 'count_sessions'].iloc[0]
sess_web =  by_device_conversion.loc[ by_device_conversion['device_type'] == 'desktop_web', 'count_sessions'].iloc[0] 
sess_ios =  by_device_conversion.loc[ by_device_conversion['device_type'] == 'ios', 'count_sessions'].iloc[0] 
sess_mweb =  by_device_conversion.loc[ by_device_conversion['device_type'] == 'mobile_web', 'count_sessions'].iloc[0] 

purch_all_device = by_device_conversion['count_purchase'].sum()
sess_all_device  = by_device_conversion['count_sessions'].sum()

mweb_stats = [purch_mweb, sess_mweb - purch_mweb]
all_device_stats = [purch_all_device, sess_all_device - purch_all_device]
device_outcomes =  np.array([all_device_stats, mweb_stats])
stats.chi2_contingency(device_outcomes)

(134.59007677364482,
 4.0588291038751217e-31,
 1,
 array([[   3515.22221864,  224048.77778136],
        [   1760.77778136,  112226.22221864]]))

In [7]:
#Conversion rate by Device

### Section 2 - Solution Summary

1) The student has identified device type and region as the actionable elements of their personas as a first step.  

2) The student has identified the item page is getting the most traffic and is the best opporunity to show traction in the short term.

3) The student has at least identified the fact that there is a large drop off in conversion for users on mobile_web as compared to other users.


## Section 3 - Generate Hypotheses and Recommended Next Step [STUDENT]

Now you're going to generate hypotheses based on your personas, journey, and risks & opportunities

Using the personas and journey steps identified above:


![screen shots of mock website icecreamsandwi.ch ](screenshots.png)

1) Find the best opportunity for your first set of hypotheses  
2) Generate hypotheses of what do do next  
3) Summarize your hypotheses and next best action for each  


## Section 3 - Generate Hypotheses and Recommended Next Step [SOLUTION]

![screen shots of mock website icecreamsandwi.ch ](screenshot_solution.png)

1) Student should be able to call out that they identified Mobile Web for All Users as the best first place to generate hypotheses.

2) Student should identify the lack of "Free Shipping" on mobile web as an opportunity 

3) Student should recommend an a/b experiment on mobile web.





